# Classificador Naive Bayes

Atualmente, o algoritmo se tornou popular na área de Aprendizado de Máquina (Machine Learning) para **categorizar textos baseado na frequência das palavras usadas**, e assim pode ser usado para identificar se determinado e-mail é um SPAM ou sobre qual assunto se refere determinado texto, por exemplo.

Por ser muito simples e rápido, possui um desempenho relativamente maior do que outros classificadores. Além disso, o Naive Bayes só precisa de um pequeno número de dados de teste para concluir classificações com uma boa precisão.

A principal característica do algoritmo, e também o motivo de receber “naive” (ingênuo) no nome, é que **ele desconsidera completamente a correlação entre as variáveis (features)**. Ou seja, se determinada fruta é considerada uma “Maçã” se ela for “Vermelha”, “Redonda” e possui “aproximadamente 10cm de diâmetro”, o algoritmo não vai levar em consideração a correlação entre esses fatores, tratando cada um de forma independente.

In [624]:
from functools import lru_cache
from nltk.corpus import floresta
from textblob.classifiers import NaiveBayesClassifier, DecisionTreeClassifier
from string import digits, punctuation
from pprint import pprint
from random import shuffle
import numpy
import stop_words
import textblob
import re
import nltk
import requests
import csv
import math

## Dados

Neste notebook vamos tentar aplicar o classificador Naive Bayes para classificar discursos dos parlamentares da Câmara dos Deputados proferidos em plenário. Os dados utilizados para análise foram obtidos através do [Babel](https://dev.babel.labhackercd.leg.br/), um repositórios de dados de manifestações políticas, e podem ser acessados pelo *endpoint*: https://dev.babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=2 .

In [625]:
response = requests.get('https://dev.babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=2')
data = response.json()['results']
response = requests.get('https://dev.babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=2&page=10')
data += response.json()['results']
response = requests.get('https://dev.babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=2&page=20')
data += response.json()['results']

speeches = []
for speech in data:
    for attr in speech['attrs']:
        if attr['field'] == 'original':
            speeches.append(attr['value'])
            break

print(len(speeches), 'discursos coletados')

600 discursos coletados


In [626]:
# Método para limpar texto do discurso tirando as notas do taquigrafo
def clear_speech(text):
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[OA] SRA?[\w\s.]+-', '', text)
    text = re.sub(r'PRONUNCIAMENTO[\sA-Z]+\s', '', text)
#     text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s[\.\"]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[Vv]\.[Ee][Xx][Aa]\.', 'v.exa', text)
    text = re.sub(r'[Aa][Rr][Tt]\.', 'art', text)
    text = re.sub(r'[Ss][Rr][Ss]?\.', 'sr', text)
    text = re.sub(r'[Ss][Rr][Aa][Ss]?\.', 'sr', text)
    text = re.sub(r'\d', '', text)
    return text.strip()

cleaned_text = clear_speech('. '.join(speeches))
blob = textblob.TextBlob(cleaned_text)

### Sentenças

Para realizar a classificação de um discurso, ele será divido em sentenças, utilizando a biblioteca TextBlob, e cada sentença será classificada individualmente e o conjunto de sentenças classificadas (uma sentença não precisa ser, necessariamente, classificada) definirá a classificação de um discurso.

In [627]:
print(blob.sentences[23], end='\n\n')
print(blob.sentences[233], end='\n\n')
print(blob.sentences[100], end='\n\n')

Para se ter ideia, na Bahia, o PSDB está com uma propaganda no seu horário na televisão na qual os Deputados do PSDB dizem que o Governo da Bahia, na área da saúde, só faz propaganda.

Esperamos, com relação a esse ponto, que a Comissão da Verdade revele os responsáveis, nestas empresas, pela cooperação com a tortura, o assassinato e outros crimes bárbaros cometidos pelo regime de exceção.

A poucos dias da Copa do Mundo, o Brasil se prepara para dar início à Copa das Copas, uma competição cercada, é verdade, por graves denúncias , que estão sendo investigadas pela Justiça e por outros órgãos responsáveis, mas que colocará o Brasil no centro das atenções mundiais por quase quarenta dias.



## Stopwords

*Stopwords* (ou palavras de parada – tradução livre) são palavras que podem ser consideradas irrelevantes para o conjunto de resultados a ser exibido em uma busca realizada em uma search engine. Exemplos: as, e, os, de, para, com, sem, foi. Esse conjunto de palavras pode variar em função do contexto em que será utilizado.

Para esse notebook serão utilizadas as palavras fornecidas pela biblioteca NLTK, além de outras palavras selecionadas de acordo com os textos utilizados.

In [628]:
stemmer = nltk.RSLPStemmer()

In [629]:
stopwords = stop_words.get_stop_words('portuguese') + [
    'presidente', ',', '.', '...', 'é', 'questão', 'art', 'ordem', 'v.exa', ':', 'governo', 'sr', 'agência', 'º',
    'aqui', 'vai', 'artigo', '§', 'neste', 'vamos', 'agora', "''", 'fazer', 'mesa', 'ainda', 'porque', 'trata',
    'estrutura', 'sobre', 'então', 'todos', 'obstrução', 'votação', 'presença', 'deputados', 'vou', 'brasil',
    'discutir', 'vigência', 'colocar', 'regimento', 'momento', ';', 'dois', 'dessa', 'medida', 'proposta',
    'casa', 'matéria', 'queria', 'assim', 'possamos', 'microfone', 'certeza', 'hoje', 'profissional', 'deixar',
    'provisória', 'ora', 'base', 'importante', 'veto', 'fala', '!', 'ministério', 'aumento', 'inciso',
    'manifestação', 'xiii', 'diálogo', 'podemos', 'apenas', 'poder', 'efeitos', 'pode', 'acordo', 'solicitação',
    'reflexão', '?', 'ausência', 'aprovada', 'lideranças', 'dizer', 'bancada', 'portanto', 'peço', 'recolher',
    'prática', 'pois', 'democracia', 'milhões', 'bilhões', 'melhoria', 'atividade', 'claro', 'saber', 'dar',
    'avanço', 'condições', 'desastre', 'especialmente', 'exatamente', 'política', 'vezes', 'fazê-lo', 'têm',
    'derrubar', 'precisa', 'custo', 'necessária', 'cláusula', 'proposição', '-', 'palavra', 'tempo', 'segundos',
    'fez', 'necessário', 'zero', 'interesse', 'srs', 'sr', 'sras', 'sra', 'deputado', 'presidente', 'é', 'nº',
    's.a.', 'v.exa.', 'v.exa', '#', 'anos', 'º', 'exa', 'mesa', 'legislatura', 'sessão', 'maioria', 'seguinte',
    'mandato', 'bilhões', 'quilômetros', 'ª', 'parabéns', 'membros', 'convido', 'usual', 'biênio',
    'brasil', 'palavra', 'discussão', 'período', 'início', 'pronunciamento', 'suplente', 'atividade', 'ação',
    'ações', 'daqueles', 'diferenças', 'pasta', 'milhares', 'srªs', 'emenda', 'àqueles', 'tamanha', 'mês',
    'capaz', 'km', 'modelo', 'tarefas', 'colegas', 'programa', 'voz', 'meios de comunicação', 'pronunciamento',
    'casa', 'sessão', 'deliberativa', 'solene', 'ordinária', 'extraordinária', 'encaminhado', 'orador', 'tv',
    'divulgar', 'deputado', 'parlamento', 'parlamentar', 'projeto', 'proposta', 'requerimento', 'destaque',
    'veto', 'federal', 'câmara', 'senado', 'congresso', 'nacional', 'país', 'estado', 'brasil', 'lei',
    'política', 'povo', 'voto', 'partido', 'liderança', 'bancada', 'bloco', 'líder', 'lider', 'frente',
    'governo', 'oposição', 'presença', 'presente', 'passado', 'ausência', 'ausencia', 'ausente', 'obstrução',
    'registrar', 'aprovar', 'rejeitar', 'rejeição', 'sabe', 'matéria', 'materia', 'questão', 'ordem', 'emenda',
    'sistema', 'processo', 'legislativo', 'plenário', 'pedir', 'peço', 'comissão', 'especial', 'permanente',
    'apresentar', 'encaminhar', 'encaminho', 'orientar', 'liberar', 'apoiar', 'situação', 'fato', 'revisão',
    'tempo', 'pauta', 'discutir', 'discussão', 'debater', 'retirar', 'atender', 'colegas', 'autor', 'texto',
    'medida', 'união', 'república', 'audiência', 'audiencia', 'público', 'publico', 'reunião', 'agradecer',
    'solicitar', 'assistir', 'contrário', 'favorável', 'pessoa', 'comemorar', 'ato', 'momento', 'diretora',
    'possível', 'atenção', 'agradeço', 'naquele', 'necessárias', 'presidenta', 'compromisso', 'geradas',
    'primeiro', 'simplesmente', 'ideal', 'argumento', 'i', 'válido', 'envolvidos', 'nesse', 'aspecto',
    'existentes', 'normativo', 'irá', 'nada', 'melhor', 'esperarmos', 'pouco', 'resolvermos', 'problema',
    'postura', 'faltas', 'declara', '%', 'grande', 'dia', 'obrigado', 'agradeço', 'agradecido', 'população',
    'maior', 'cada', 'bem', 'mundo', 'desta', 'mil', 'sendo', 'outros', '$', '!', '@', '#', '&', '(', ')', 'sim',
    'r', 'sempre', 'além', 'semana', 'relação', 'onde', 'meio', 'inclusive', 'lá', 'vem', 'menos', 'menor',
    'qualquer', 'desde', 'ontem', 'hoje', 'exemplos', 'exemplo', 'tão', 'fim', 'janeiro', 'fevereiro', 'março',
    'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro', 'alguns', 'tudo',
    'durante', 'gostaria', 'três', 'conta', 'feito', 'através', 'antes', 'depois', 'verdade', 'bom', 'quase',
    'setor', 'aí', 'disse', 'principalmente', 'final', 'vão', 'coisa', 'ver', 'sentido', 'nova', 'vários', 'novo',
    'nenhuma', 'quanto', 'infelizmente', 'felizmente', 'número', 'duas', 'dois', 'tanto', 'acho', 'achar',
    'enquanto', 'deve', 'apelo', 'papel', 'últimos', 'faço', 'fazer', 'garantir', 'garantia', 'fica', 'obrigado',
    'obrigado..', 'assunto', 'sido', 'vir', 'incrementar', 'central', 'aproximado', 'aproximadamente',
    'hipotética', 'hipotese', 'hipótese', 'média', 'superior', 'superiores', 'gerais', 'venha', 'minas'
]
stopwords += [x for x in punctuation]
stem_stopwords = [stemmer.stem(word) for word in stopwords]

In [630]:
@lru_cache()
def simplify_tag(tag):
    if "+" in tag:
        return tag[tag.index("+") + 1:]
    else:
        return tag

floresta_twords = floresta.tagged_words()
for (word, tag) in floresta_twords:
    tag = simplify_tag(tag)
    words = word.casefold().split('_')
    if tag not in ('adj', 'n', 'prop', 'nprop', 'est', 'npro'):
        stopwords += [stemmer.stem(word) for word in words]

stopwords = list(set(stopwords))

In [631]:
all_words = nltk.tokenize.word_tokenize(cleaned_text)
words = [word.casefold() for word in all_words if stemmer.stem(word) not in stem_stopwords]
dist = nltk.FreqDist(words)
print('Palavras mais comuns')
print(dist.most_common(100))

Palavras mais comuns
[('trabalho', 206), ('saúde', 175), ('rio', 164), ('cidade', 139), ('vida', 135), ('sociedade', 132), ('ministro', 130), ('dilma', 123), ('trabalhadores', 120), ('polícia', 110), ('paulo', 105), ('segurança', 103), ('recursos', 102), ('educação', 102), ('justiça', 100), ('mulheres', 96), ('michel', 93), ('reais', 90), ('direito', 89), ('respeito', 87), ('presidente..', 86), ('crise', 85), ('social', 85), ('empresas', 84), ('tribuna', 82), ('prefeito', 82), ('pt', 81), ('dinheiro', 80), ('economia', 79), ('municípios', 79), ('município', 78), ('violência', 78), ('reforma', 77), ('servidores', 77), ('sul', 73), ('constituição', 73), ('crime', 73), ('urgência', 72), ('direitos', 71), ('área', 70), ('desenvolvimento', 68), ('pec', 68), ('maneira', 67), ('defesa', 67), ('corrupção', 67), ('nome', 64), ('oportunidade', 63), ('luta', 62), ('tribunal', 61), ('petrobras', 61), ('posição', 60), ('história', 60), ('bahia', 56), ('serviço', 56), ('dentro', 55), ('lula', 54), (

## Naive Bayes

Para esse notebook, usaremos a implementação do TextBlob para classificar as sentenças. No TextBlob, podemos definir um método de extração de atributos (*feature extraction*) que são utilizados para comparar dois textos. Esse método deve receber como parâmetro o texto, pode receber um segundo argumento que contem os dados de treinamento e deve retornar um dicionário de atributos do texto recebido

In [632]:
regex = re.compile('[%s]' % re.escape(punctuation))

def freq_feature_extractor(document):
    document = regex.sub(' ', document.casefold())
    tokens = nltk.tokenize.word_tokenize(document)
    tokens = [stemmer.stem(token) for token in tokens if stemmer.stem(token) not in stem_stopwords]
    dist = nltk.FreqDist(tokens)
    
    features = {
        stem: True
#         stem: dist.freq(stem)
        for stem, _ in dist.most_common()
    }
    return features

freq_feature_extractor('Calcula-se que a taxação de grandes patrimônios poderia render aproximadamente  bilhões de reais por ano se aplicada uma alíquota média de % sobre os bens das pessoas, em uma simulação hipotética, sobre valores superiores a  milhão de reais.')

{'real': True,
 'calcul': True,
 'tax': True,
 'patrimôni': True,
 'rend': True,
 'aplic': True,
 'alíquot': True,
 'simul': True,
 'val': True}

## Dados de treinamento

A planilha de treinamento está disponível no [Google Drive](https://docs.google.com/spreadsheets/d/1qmJjRexlSUYOAN12IY_82NyOUaLZlbBsoeYRuulEjFE/edit?usp=sharing).

In [633]:
LABELS_RELATION = {
    0: 'none',
    1: 'adm-publica',
    2: 'agricultura-pecuaria-pesca-extrativismo',
    3: 'arte-cultura-religiao',
    4: 'cidades-desenvolvimento-urbano',
    5: 'ciencia-tecnologia-inovacao',
    6: 'ciencias-exatas',
    7: 'ciencias-sociais',
    8: 'comunicacoes',
    9: 'defesa-seguranca',
    10: 'direito-civil',
    11: 'direito-constitucional',
    12: 'direito-consumidor',
    13: 'direito-justica',
    14: 'direito-penal',
    15: 'direitos-humanos-minorias',
    16: 'economia',
    17: 'educacao',
    18: 'energia-recursos-hidricos-minerais',
    19: 'esporte-lazer',
    20: 'estrutura-fundiaria',
    21: 'financas-publicas-orcamento',
    22: 'homenagens-datas-comemorativas',
    23: 'industria-comercio-servicos',
    24: 'meio-ambiente-desenvolvimento-sustentavel',
    25: 'politica-partidos-eleicoes',
    26: 'previdencia-assistencia-social',
    27: 'processo-legislativo-atuacao-parlamentar',
    28: 'relacoes-internacionais-comercio-exterior',
    29: 'saude',
    30: 'trabalho-emprego',
    31: 'turismo',
    32: 'viacao-transporte-mobilidade',
}

In [634]:
theme_data = []
content_data = []

with open('naive-bayes-train.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    next(reader)
    for row in reader:
        for idx, category in enumerate(row[1:-1]):
            if category != '' and idx > 0:
                theme_data.append((row[0], LABELS_RELATION[idx]))
                content_data.append((row[0], 'content'))
            elif category != '' and idx == 0:
                content_data.append((row[0], 'useless'))

shuffle(content_data)
content_data_size = len(content_data)
content_train = content_data[:math.floor(content_data_size * 0.7)]
content_test = content_data[math.floor(content_data_size * 0.7):]
print('Treinamento de conteúdo: {} sentenças'.format(len(content_train)))
print('Teste de conteúdo: {} sentenças'.format(len(content_test)))

shuffle(theme_data)
theme_data_size = len(theme_data)
theme_train = theme_data[:math.floor(theme_data_size * 0.7)]
theme_test = theme_data[math.floor(theme_data_size * 0.7):]
print('Treinamento de temas: {} sentenças'.format(len(theme_train)))
print('Teste de temas: {} sentenças'.format(len(theme_test)))

Treinamento de conteúdo: 708 sentenças
Teste de conteúdo: 304 sentenças
Treinamento de temas: 510 sentenças
Teste de temas: 219 sentenças


In [635]:
content_classifier = NaiveBayesClassifier(content_train, feature_extractor=freq_feature_extractor)
content_classifier.accuracy(content_test)

0.7105263157894737

In [636]:
theme_classifier = NaiveBayesClassifier(theme_train, feature_extractor=freq_feature_extractor)
theme_classifier.accuracy(theme_test)

0.0319634703196347

In [637]:
theme_classifier = DecisionTreeClassifier(theme_train, feature_extractor=freq_feature_extractor)
theme_classifier.accuracy(theme_test)

0.2557077625570776

### Divisão dos macro-temas

Para melhorar a acurácia do classificador, os 32 temas serão agrupados em macrotemas, para facilitar a classificação.

In [638]:
MACRO_THEMES_RELATION = {
    'cidades-desenvolvimento-urbano': 'cidades-transportes',
    'viacao-transporte-mobilidade': 'cidades-transportes',
    
    'comunicacoes': 'ct-comunicacoes',
    'ciencia-tecnologia-inovacao': 'ct-comunicacoes',
    'ciencias-exatas': 'ct-comunicacoes',
    
    'defesa-seguranca': 'seguranca',
    
    'previdencia-assistencia-social': 'trabalho-previdencia-assistencia',
    'trabalho-emprego': 'trabalho-previdencia-assistencia',
    
    'relacoes-internacionais-comercio-exterior': 'relacoes-exteriores',
    
    'saude': 'saude',
    
    'arte-cultura-religiao': 'educacao-cultura-esporte',
    'educacao': 'educacao-cultura-esporte',
    'esporte-lazer': 'educacao-cultura-esporte',
    'turismo': 'educacao-cultura-esporte',
    
    'homenagens-datas-comemorativas': 'politica-adm-publica',
    'politica-partidos-eleicoes': 'politica-adm-publica',
    'processo-legislativo-atuacao-parlamentar': 'politica-adm-publica',
    'adm-publica': 'politica-adm-publica',
    
    'ciencias-sociais': 'direitos-humanos',
    'direitos-humanos-minorias': 'direitos-humanos',
    
    'agricultura-pecuaria-pesca-extrativismo': 'agropecuaria',
    'estrutura-fundiaria': 'agropecuaria',
    
    'financas-publicas-orcamento': 'economia',
    'economia': 'economia',
    
    'meio-ambiente-desenvolvimento-sustentavel': 'meio-ambiente-energia',
    'energia-recursos-hidricos-minerais': 'meio-ambiente-energia',
    
    'direito-consumidor': 'consumidor',
    'industria-comercio-servicos': 'consumidor',
    
    'direito-civil': 'justica',
    'direito-constitucional': 'justica',
    'direito-justica': 'justica',
    'direito-penal': 'justica',
}

In [639]:
macrotheme_data = [(sentence, MACRO_THEMES_RELATION[label]) for sentence, label in theme_data]
shuffle(macrotheme_data)
macrotheme_data_size = len(macrotheme_data)
macrotheme_train = macrotheme_data[:math.floor(macrotheme_data_size * 0.7)]
macrotheme_test = macrotheme_data[math.floor(macrotheme_data_size * 0.7):]
print('Treinamento de macro temas: {} sentenças'.format(len(macrotheme_train)))
print('Teste de macro temas: {} sentenças'.format(len(macrotheme_test)))

Treinamento de macro temas: 510 sentenças
Teste de macro temas: 219 sentenças


In [640]:
macrotheme_classifier = DecisionTreeClassifier(macrotheme_train, feature_extractor=freq_feature_extractor)
macrotheme_classifier.accuracy(macrotheme_test)

0.365296803652968

In [641]:
print(len(macrotheme_classifier.labels()), ' Categorias:')
print(macrotheme_classifier.labels())

13  Categorias:
['ct-comunicacoes', 'consumidor', 'agropecuaria', 'direitos-humanos', 'trabalho-previdencia-assistencia', 'seguranca', 'politica-adm-publica', 'meio-ambiente-energia', 'cidades-transportes', 'educacao-cultura-esporte', 'justica', 'saude', 'economia']


In [642]:
MACRO_THEMES_TRAIN = {
    'ct-comunicacoes': [],
    'consumidor': [],
    'agropecuaria': [],
    'direitos-humanos': [],
    'saude': [],
    'seguranca': [],
    'politica-adm-publica': [],
    'meio-ambiente-energia': [], 
    'cidades-transportes': [],
    'educacao-cultura-esporte': [],
    'justica': [],
    'trabalho-previdencia-assistencia': [],
    'economia': [],
    'relacoes-exteriores': [],
}

for sentence, label in theme_data:
    macro_theme = MACRO_THEMES_RELATION[label]
    train_list = MACROTHEMES_TRAIN.get(macro_theme, [])
    train_list.append((sentence, label)) 
    MACRO_THEMES_TRAIN[macro_theme] = train_list

In [643]:
MACRO_THEMES_CLASSIFIERS = {
    'ct-comunicacoes': None,
    'consumidor': None,
    'agropecuaria': None,
    'direitos-humanos': None,
    'saude': None,
    'seguranca': None,
    'politica-adm-publica': None,
    'meio-ambiente-energia': None, 
    'cidades-transportes': None,
    'educacao-cultura-esporte': None,
    'justica': None,
    'trabalho-previdencia-assistencia': None,
    'economia': None,
    'relacoes-exteriores': None,
}

for macrotheme in MACRO_THEME_CLASSIFIERS.keys():
    macrotheme_data = MACRO_THEMES_TRAIN[macrotheme]

    shuffle(macrotheme_data)
    macrotheme_data_size = len(macrotheme_data)
    macrotheme_train = macrotheme_data[:math.floor(macrotheme_data_size * 0.7)]
    macrotheme_test = macrotheme_data[math.floor(macrotheme_data_size * 0.7):]

    if len(macrotheme_train):
        print('Training', macrotheme, 'classifier with', len(macrotheme_train), 'sentences')
        macrotheme_classifier = NaiveBayesClassifier(macrotheme_train)
        MACRO_THEMES_CLASSIFIERS[macrotheme] = macrotheme_classifier
        print('Training finished! Testing with', len(macrotheme_test), '. Accuracy: ', macrotheme_classifier.accuracy(macrotheme_test), end='\n\n')
    
    

Training ct-comunicacoes classifier with 35 sentences
Training finished! Testing with 16 . Accuracy:  1.0

Training consumidor classifier with 31 sentences
Training finished! Testing with 14 . Accuracy:  1.0

Training agropecuaria classifier with 98 sentences
Training finished! Testing with 43 . Accuracy:  0.9767441860465116

Training direitos-humanos classifier with 134 sentences
Training finished! Testing with 58 . Accuracy:  0.8620689655172413

Training saude classifier with 54 sentences
Training finished! Testing with 24 . Accuracy:  1.0

Training seguranca classifier with 90 sentences
Training finished! Testing with 39 . Accuracy:  1.0

Training politica-adm-publica classifier with 503 sentences
Training finished! Testing with 217 . Accuracy:  0.9032258064516129

Training meio-ambiente-energia classifier with 96 sentences
Training finished! Testing with 42 . Accuracy:  0.7857142857142857

Training cidades-transportes classifier with 52 sentences
Training finished! Testing with 23 